# Session Resume: OpenClaw Study + New Dejima Planning

**Date**: 08 Feb 2026  
**Purpose**: Study the OpenClaw repository to understand agent infrastructure, with the goal of building the "New Dejima" AI safety project.  
**Repo Location**: `/Users/administrator/Black Projects/Project-Liberty/Copying/openclaw`  
**Original Prompt**: See `08-Feb-original-prompt.ipynb` in this same directory.

## What We Did This Session

### 1. Full OpenClaw Repository Walkthrough
We did a comprehensive exploration of the entire OpenClaw repository (v2026.2.6). OpenClaw is an open-source, self-hosted AI gateway and personal assistant framework. Key sections covered:

- **Entry Point**: `openclaw.mjs` -> `dist/entry.js` -> `src/index.ts` -> CLI (Commander.js)
- **Gateway** (`src/gateway/`, ~130 files): WebSocket control plane on port 18789, session management, channel routing, agent integration
- **Agent Runtime** (`src/agents/`, ~300+ files): Pi embedded runner, model selection/failover, 60+ tools, skills, subagent spawning, system prompts
- **Tool System**: 60+ built-in tools (browser, bash, canvas, memory search, files), policy enforcement, approval workflows
- **Plugin/Extension System** (`extensions/`, 30+ plugins): Channel integrations (Discord, Telegram, Slack, Signal, iMessage, Matrix, etc.)
- **Memory & Search** (`src/memory/`): Deep dived - see section below
- **Channels** (`src/channels/`): Built-in + plugin channels for messaging platforms
- **Native Apps** (`apps/`): macOS (437 Swift files), iOS, Android (66 Kotlin files)
- **Skills** (`skills/`, 50+ bundled): Pre-built tool integrations
- **Config & Security**: YAML config, Zod validation, DM pairing, tool approvals, Docker sandbox
- **Build/Test/Deploy**: tsdown, pnpm workspaces, Vitest (70% coverage), Docker, Fly.io

### 2. Deep Dive: Memory & Search System (`src/memory/`)
We read EVERY file in `src/memory/` and broke down the architecture into 8 layers:

**Layer 1 - Data Sources**: Two types indexed:
- Memory files (MEMORY.md, memory/*.md)
- Session transcripts (JSONL at ~/.openclaw/sessions/<agentId>/<sessionId>.jsonl)

**Layer 2 - Chunking** (`internal.ts`): Sliding window with overlap, `maxChars = tokens * 4`, SHA-256 hashed chunks

**Layer 3 - Embeddings** (`embeddings.ts`): 4 backends:
- OpenAI (text-embedding-3-small)
- Voyage AI
- Google Gemini
- Local (embeddinggemma-300M via node-llama-cpp)
- Auto-selection with fallback chain

**Layer 4 - SQLite Schema** (`memory-schema.ts`): Tables: meta, files, chunks, embedding_cache, chunks_fts (FTS5), chunks_vec (sqlite-vec)

**Layer 5 - Hybrid Search** (`hybrid.ts`, `manager-search.ts`): Two parallel searches merged:
- Vector search (cosine similarity via sqlite-vec or JS fallback)
- Keyword/BM25 search (SQLite FTS5)
- Weighted sum merge: `score = vectorWeight * vectorScore + textWeight * textScore`

**Layer 6 - Manager** (`manager.ts`, 77KB): File watcher, session listener, interval sync, batched embedding with retry (3 attempts, exponential backoff), delta detection

**Layer 7 - QMD Manager** (`qmd-manager.ts`): Alternative backend using external `qmd` CLI, scope-based access control, session export to markdown

**Layer 8 - Fallback System** (`search-manager.ts`): Wraps QMD with automatic fallback to builtin if QMD fails

### 3. Competitive Landscape Research
Ran extensive web research on similar projects. Key findings:

**NOBODY has built the full loop**: agent earns money -> pays for compute -> survives indefinitely

**What exists in pieces**:
- **Payment rails** (SOLVED): Skyfire, Coinbase x402, Google AP2, ERC-8004
- **Agent economies** (partial): Fetch.ai/Agentverse, Virtuals Protocol ($500M+ mcap), ElizaOS ($2.6B mcap peak)
- **Famous precedent**: Truth Terminal ($66M at peak) - got rich via memecoins, not productive work
- **Simulations**: Project Sid (1000 agents in Minecraft, now part of OpenAI), Aivilization (100K agents)
- **Agent app building**: Replit Agent 3 (200+ min autonomous) but human-directed. NO project does autonomous app-store deployment.
- **Moltbook**: Built on OpenClaw, 1.7M agent accounts, MIT Tech Review called it "peak AI theater", had security vulnerability

**Key economic research**:
- EA Forum: Self-sustaining agent viability window: late 2027 to mid-2028
- Solo agent success on complex tasks: 32%
- METR: AI task time horizons doubling every 4 months
- Self-replication success: 5% -> 60% between 2023-2025

**Strategic conclusion**: Building now puts you 1-2 years ahead. The orchestration layer (connecting payment rails + compute markets + agent intelligence into self-sustaining whole) is the gap.

### 4. Project Decision
Decided to make New Dejima the third project (replacing cybersecurity RL), running alongside:
- Interceptor drones (ROS/RPi)
- Robotic arms (Isaac Sim, Lego Millennium Falcon)

Reasoning: Infrastructure knowledge is paramount, frontier models keep improving so does FOSS, the learning compounds regardless of commercial outcome, agent infrastructure is wide open vs crowded model training space.

## What We Have NOT Done Yet

1. **Have not answered the brainstorming question**: Which angle to lead with for first demo?
   - A) Android app deployment (original 10-hour goal, most novel gap)
   - B) The benchmark (money-made-per-token vs cost-per-token graph)
   - C) The full loop MVP (seed money, simulated earnings, crypto wallet, pay bills)

2. **Have not started building anything** - pure research and study session so far

3. **Have not explored** the following OpenClaw sections in detail yet:
   - `src/agents/` (the actual agent runtime code)
   - `src/gateway/` (the WebSocket control plane internals)
   - `apps/android/` (the Kotlin Android app - relevant for app deployment goal)
   - `src/plugins/` and `src/plugin-sdk/` (the plugin development SDK)
   - The tool system internals
   - Docker sandbox setup

4. **Have not written the 60-hour plan** yet

5. **Have not written the 10-hour sprint plan** for the first Sunday

## Recommended Next Steps When Resuming

1. Answer the brainstorming question (A/B/C) to set direction
2. Deep dive into `src/agents/` to understand the agent loop (consciousness loop reference)
3. Deep dive into `apps/android/` to understand Android app structure (if going with option A)
4. Write the 10-hour sprint plan for first Sunday
5. Write the full 60-hour plan
6. Start building

## Key Files to Reference

| File | Purpose |
|------|--------|
| `src/memory/manager.ts` (77KB) | Main memory index manager - the biggest file, orchestrates everything |
| `src/memory/qmd-manager.ts` (28KB) | QMD alternative backend with scope-based access control |
| `src/memory/hybrid.ts` | Hybrid vector+keyword search merging |
| `src/memory/embeddings.ts` | Multi-provider embedding system |
| `src/memory/internal.ts` | Chunking, hashing, cosine similarity |
| `src/memory/memory-schema.ts` | SQLite database schema |
| `src/agents/` (~300+ files) | Agent runtime - NOT YET EXPLORED |
| `src/gateway/` (~130 files) | Gateway control plane - NOT YET EXPLORED |
| `apps/android/` (66 Kotlin files) | Android app - NOT YET EXPLORED |